In [ ]:
# default_exp model.module

# babilim.model.module

> The base class for every module.

This code is under the MIT License.

In [3]:
# export
# MIT License
#
# Copyright (c) 2019 Michael Fuerst
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

In [4]:
#export
from typing import Sequence, Any, Sequence, Callable, Dict, Iterable
from collections import defaultdict
import inspect

import numpy as np

import babilim
from babilim import PYTORCH_BACKEND, TF_BACKEND, warn
from babilim.core import StatefullObject, RunOnlyOnce, Tensor
from babilim.data import Dataset

#hide

# Module

A module is the base building block of all models.

In [30]:
#export
class Module(StatefullObject):
    def __init__(self, layer_type: str):
        """
        A module is the base building block of all models and layers.

        Users must overwrite `__init__`,`call` and `build`. For the latter do not forget the `@RunOnlyOnce`-Annotation.
        The rest comes pre implemented and should not be overwritten.
        When overwriting the `__init__` call the super init: `super().__init__(layer_type)`.

        :param layer_type: The type of the layer provided.
        """
        super().__init__()
        self.initialized_module = False
        self.__layer_type = layer_type

    def __call__(self, *args, **kwargs) -> Any:
        """
        Makes a module callable. Automatically wraps tensorflow or pytorch tensors to ITensors from babilim.

        <code>module = MyModule()</code>

        <code>module(*args, **kwargs)</code>

        Warning: This function should not be overwritten. Instead overwrite `call` with no underscores.

        :param *args: All indexed parameters of your call function derivate.
        :param **kwargs: All named parameters of your call function derivate.
        """
        # ensure that call gets called with ITensor objects but the caller can use native tensors.
        args, wrapped_args = self._wrapper.wrap(args)
        kwargs, wrapped_kwargs = self._wrapper.wrap(kwargs)
        self.build(*args, **kwargs)
        result = self.call(*args, **kwargs)
        parent_dict = inspect.stack()[1][0].f_locals
        if "self" in parent_dict:
            parent = parent_dict["self"]
            self._register_params(parent)
        if wrapped_args or wrapped_kwargs:
            return self._wrapper.unwrap(result)
        else:
            return result

    def initialize(self, dataset: Dataset):
        """
        Initializes your module by running a sample of your dataset through it.

        :param dataset: The dataset you want to use for initialization. (Must be of type babilim.data.Dataset)
        """
        if not self.initialized_module:
            if babilim.DEBUG_VERBOSITY:
                babilim.info("Build Model")
            self.initialized_module = True
            dataloader = dataset.to_dataloader()
            features, _ = next(iter(dataloader))
            self(**features._asdict())

    def predict(self, **kwargs):
        """
        Pass in single training examples as numpy arrays.
        And predict the value without gradients.
        Should be used for testing and evaluation.

        If your network has eval modes you need to set them manually.

        The array must not have batch dimension.

        :param kwargs: The parameters to feed the network as a single example.
        :return: The output for a single example.
        """
        kwargs = {k: np.array([kwargs[k]]) for k in kwargs.keys() if isinstance(kwargs[k], np.ndarray)}
        kwargs = {k: Tensor(data=kwargs[k], trainable=False) for k in kwargs.keys()}

        preds = self.__call__(**kwargs)
        tmp = preds._asdict()
        tmp = {k: tmp[k].numpy()[0] for k in tmp.keys()}
        preds = type(preds)(**tmp)
        return preds

    def build(self, *args, **kwargs) -> None:
        """
        This function will build your model and must be annotated with the RunOnlyOnce-Annotation.

        Allocating weight tensors should be done here.
        You can make use of the knowledge of your inputs to compute shapes for your weight tensors.
        This will make coding dimensions a lot easier.

        <code> @RunOnlyOnce</code>

        <code>def build(self, image: ITensor) -> None:</code>

        :param *args: You must specify the exact same parameters as for your call.
        :param **kwargs: You must specify the exact same parameters as for your call.
        """
        pass

    def call(self, *args, **kwargs) -> Any:
        """
        Makes a module callable and contains the forward pass of your model.
        This should be pure computation and not allocate any weights.
        Allocating weights should be done in the `build` function.

        This function gets called by `__call__` and must be overwritten by any derived class.

        <code>def call(self, image: ITensor) -> NetworkOutput:</code>

        :param *args: You can specify any parameters you want.
        :param **kwargs: You can specify any named parameters you want.
        """
        raise NotImplementedError("Every modules must implement this method.")

    @property
    def layer_type(self):
        return self.__layer_type

    @property
    def submodules(self):
        modules = []
        for k in self.__dict__:
            v = self.__dict__[k]
            if isinstance(v, Module):
                modules.append(v)
                modules.append(v.submodules)
        return modules

    def modules(self):
        """
        Returns an iterator over all modules in the network.
        """
        for name, module in self.named_modules():
            yield module

    def named_modules(self, memo=None, prefix=''):
        """
        A named list of all modules.
        """
        modules = {}
        for k in self.__dict__:
            v = self.__dict__[k]
            if isinstance(v, Module):
                modules[prefix + "." + k] = v
                modules.update(**dict(v.named_modules(memo, prefix)))
        return list(modules.items())

    @RunOnlyOnce
    def _register_params(self, module):
        """
        Allows registration of the parameters with a native module.

        This makes the parameters of a babilim modules available to the native modules.
        When using a babilim modules in a native modules, use this function and pass the native module as a parameter.

        This function works by adding all trainable_variables to the module you pass.
        Warning: You need to build the babilim modules before calling this function. Building can be done by calling for example.

        Here is a pytorch example:

        .. code-block:: python

            import torch
            from torch.nn import Module
            from babilim.modules import Linear


            class MyModule(Module):
                def __init__(self):
                    super().__init__()
                    self.linear = Linear(10)

                def forward(self, features):
                    result = self.linear(features)
                    self.linear.register_params(self)
                    return result

        :param module: The native module on which parameters of this modules should be registered.
        """
        if babilim.is_backend(PYTORCH_BACKEND):
            from torch.nn import Module
            if isinstance(module, Module):
                myname = "_error_"
                for var in module.__dict__:
                    if module.__dict__[var] == self:
                        myname = var
                    if isinstance(module.__dict__[var], list) and self in module.__dict__[var]:
                        myname = "{}/{}".format(var, module.__dict__[var].index(self))

                # Register self as pytorch module.
                module._modules[myname] = self

                for name, param in self.named_variables.items():
                    if param.trainable:
                        module.register_parameter(myname + name, param.native)
                    else:
                        module.register_buffer(myname + name, param.native)
        else:
            warn("Not implemented for tf2 but I think it is not required.")